In [225]:
needs = [8000, 13000,23000,34000,10000,18000,23000,38000,12000,13000,32000,41000]

In [227]:
from collections.abc import Sequence

# Moving Average Methods

In [59]:
def moving_average_methods(
    data: Sequence[float], window_size: int
)->list[float | None]:
    
    if window_size < 1:
        raise ValueError
        
    if window_size > len(data):
        raise ValueError

    levels: list[float | None] = []
    
    for i in range(len(data)):
        if i < window_size - 1:
            levels.append(None)
        else:
            window = data[i-window_size+1 : i+1]
            level_value = sum(window)/window_size
            levels.append(level_value)
    return levels

In [61]:
moving_average_methods(needs, 4)

[None,
 None,
 None,
 19500.0,
 20000.0,
 21250.0,
 21250.0,
 22250.0,
 22750.0,
 21500.0,
 23750.0,
 24500.0]

# Exponential Smoothing Methods

In [143]:
def exponential_smoothing_methods(
    data: Sequence[float], alpha: float = None
)-> list[float]:

    if alpha == None:
        alpha = 2 / ( 1 + len(data))
        print(f"Set alpha: {alpha}")

    levels = []
    
    first_level = sum(data)/len(data)
    levels.append(first_level)
    
    for i in range(len(data)):
        level_value = alpha * data[i] + ( 1 - alpha ) * levels[i]
        levels.append(level_value)

    return levels
    

In [145]:
exponential_smoothing_methods(needs)

Set alpha: 0.15384615384615385


[22083.333333333332,
 19916.666666666664,
 18852.5641025641,
 19490.631163708087,
 21722.841753906843,
 19919.327637921175,
 19624.04646285638,
 20143.423930109242,
 22890.589479323207,
 21215.114174811944,
 19951.250455610105,
 21804.904231670087,
 24757.995888336227]

# Holt Model

In [217]:
def betas(data):
    Xbar = sum(range(len(data)+1))/len(data)
    Ybar = sum(data)/len(data)

    X_minus_Xbar = [x - Xbar for x in range(1, len(data)+1)]
    Y_minus_Ybar = [y - Ybar for y in data]

    X_minus_Xbar_square = [x**2 for x in X_minus_Xbar]
    X_minus_Xbar_times_y_minus_Ybar = [x*y for x, y in zip(X_minus_Xbar, Y_minus_Ybar)]

    b1 = sum(X_minus_Xbar_times_y_minus_Ybar)/ sum(X_minus_Xbar_square)

    b0 = Ybar - b1 * Xbar
    return b0, b1

def holt_model(
    data: Sequence[float], 
    alpha: float, 
    beta: float, 
)->list[float]:

    level0, trend0 = betas(data)
    forecast0 = level0 + trend0
    
    levels = []
    trends = []
    forecasts = []

    levels.append(level0)
    trends.append(trend0)
    forecasts.append(forecast0)

    for i in range(len(data)):
        level_value = alpha*data[i] + (1-alpha)*(levels[i]+trends[i])
        levels.append(level_value)
        trend_value = beta*(levels[i+1]-levels[i])+(1-beta)*trends[i]
        trends.append(trend_value)
        forecast_value = level_value + trend_value
        forecasts.append(forecast_value)

    return forecasts
    

In [219]:
holt_model(needs, 0.1, 0.2)

[13564.102564102563,
 14445.361305361304,
 15709.586946386948,
 17993.198284382288,
 21468.58452289045,
 21967.060447089985,
 23136.34756992777,
 24685.97902908322,
 27846.927761741463,
 27774.84306589905,
 27514.4699783229,
 29269.844799937906,
 31984.285243392653]

# Winter Model

In [285]:
def winter_model(
    data: Sequence[float],
    P: int,
    alpha: float, 
    beta: float, 
    gamma: float
)->list[float]:
    
    level0, trend0 = betas(data)

    # raise ValueError needs here
    r = len(data)//P

    # Preparation to calculate initial seasonal indexs
    seasonal_indexs = []
    for i in range(r):
        blocks = needs[i*P:i*P+P]
        for j in blocks:
            seasonal_index = j / (sum(blocks)/len(blocks))
            seasonal_indexs.append(seasonal_index)

    # Calculate initial seasonal indexs 1 ~ P
    for i in range(P):
        initial_seasonal_index = sum(seasonal_indexs[i::4])/3
        seasonal_indexs[i] = initial_seasonal_index
    
    # Calculate levels and Trends
    levels = []
    levels.append(level0)
    trends = []
    trends.append(trend0)

    for i in range(len(data)):
        level_value = alpha*(data[i]/seasonal_indexs[i])+(1-alpha)*(levels[i]+trends[i])
        levels.append(level_value)
        
        trend_value = beta*(levels[i+1]-levels[i])+(1-beta)*trends[i]
        trends.append(trend_value)

    # Calculate Seasonal Indexs
    seasonals = seasonal_indexs[:P]

    for i in range(len(data)-P):
        seasonal_value = gamma*(data[i]/levels[i+1])+(1-gamma)*seasonals[i]
        seasonals.append(seasonal_value)

    print(seasonals)

    # Predict
    forecasts = []

    for i in range(len(data)):
        forecast_value = (levels[i]+trends[i])*seasonals[i]
        forecasts.append(forecast_value)

    return forecasts
        
    

    
    

In [287]:
winter_model(needs, 4, 0.05, 0.1, 0.1)

[0.44983017695698274, 0.6687558918698565, 1.1731058312117701, 1.7083080999613907, 0.46292287035514973, 0.6854822867228285, 1.189022950884301, 1.717197094366313, 0.46514471334711593, 0.6979048385220659, 1.1668928985877436, 1.6961537286301545]


[6101.542656672919,
 10262.176117674944,
 20107.494637957683,
 32229.842101184786,
 9502.126081518018,
 15237.72389181452,
 28354.625003502842,
 43575.447287654846,
 12471.812944905443,
 19735.839100777514,
 34601.226235235226,
 52464.98913061365]